# Problema

Utilizar la tabla enasama_W18_0429_0505_ori_ver_anali_prelimi_estrato_reg.xlsx y
con ella realizar las siguientes activiades:

A. Calcular el nivel educativo de las personas encuestadas por edad, sexo a
nivel nacional, estatal y regional.

B. Calcular la edad promedio de inicio de consumo de alcohol edad, sexo a
nivel nacional, estatal y regional considerando el diseño muestral (puedes
usar factores de expansión igual a 1 por ahora).

C. Realizar la propuesta lógica (puede ser pseudo código o bullets explicando
cada acción) de creación de una función que realice el cálculo de promedio
de variables cuantitativas, puedes tomar como ejemplo el inciso anterior.

In [20]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.insert(0,os.path.join('..', '..'))
from procesamientos.scripts.func_analisis import *
from procesamientos.scripts.func_transformacion import *
from procesamientos.scripts.func_salida import *

In [21]:
# carga de bases de datos
nombre_archivo='enasama_W18_0429_0505_ori_ver_anali_prelimi_estrato_reg.xlsx'
db=pd.read_excel(os.path.join('..','..','datos','limpios',nombre_archivo), engine='openpyxl')

In [40]:
# selección de cuestionarios con folios válidos y que dieron autorización
b=db[(db['folio_val_dai']) & (db['cumple_autorizacion']=='SI')]

# asegurarse de que la clave de entidad sea un str
b['clave_entidad_dai']= b['clave_entidad_dai'].astype(str).str.zfill(2)

# cambio de variables con valores Y y N por 1 y 2 
b.replace({'Y':1, 'N':2}, inplace= True)

# creación de variable con nombre sexo
b.insert(0,'sexo',b['ds1'].replace({'F':'Mujeres','M':'Hombres'}))

#creación de variable llamada 'entidad'
b.insert(0,'entidad',b['nom_ent_largo'])

#creación de variable llamada 'region'
b.insert(0,'region',b['nom_region']) 

# cambio de nombre de upm 
b.rename(columns = {'upm_dai':'upm'}, inplace= True)

# creación de factor de expansión 
b.insert(0,'factor_exp',1)

# creación de grupos etarios: menores y mayores de edad
grupo_etario(b,'ci1',gps=['12-17','18-34','35-59','60-75']).head(2)
#grupo_etario(b,'ci1',gps=['12-17', '18-75']).head(2)

,factor_exp,region,entidad,sexo,upm,id,submitdate,startlanguage,startdate,datestamp,...,a1,a2,a3,a4,a5,a6,a7,aviso,cumple_autorizacion,grupo_etario
0,1,Centro Sur,Michoacán,Mujeres,2146,0,2023-10-24 05:00:00,es-MX,2023-10-23 05:00:00,2022-10-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,60-75 años
1,1,Centro Sur,Michoacán,Mujeres,2146,165,2023-10-24 05:00:00,es-MX,2023-10-23 05:00:00,2022-10-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,35-59 años


In [48]:
nivel_educativo_map = {
    1: 'Primaria incompleta (1 a 5 años)',
    2: 'Primaria completa (6 años)',
    3: 'Secundaria incompleta (1 a 2 años)',
    4: 'Secundaria completa o equivalente (3 años)',
    5: 'Bachillerato incompleto (1 a 2 años)',
    6: 'Bachillerato completo o equivalente (aprox. 3 años)',
    7: 'Estudios Universitarios incompletos (1 a 3 años)',
    8: 'Estudios Universitarios completos (4 a 5 años)',
    9: 'Estudios de Posgrado (2 a 4 años)',
    99: 'No sabe/No contesta'
}

# Nacional

In [54]:
b['nivel_educativo'] = b['ds10'].map(nivel_educativo_map)

agrupado = b.groupby(['sexo', 'grupo_etario', 'nivel_educativo']).size().reset_index(name='conteo')

agrupado['porcentaje'] = (agrupado['conteo'] / agrupado['conteo'].sum()) * 100

agrupado

,sexo,grupo_etario,nivel_educativo,conteo,porcentaje
0,Hombres,12-17 años,Bachillerato completo o equivalente (aprox. 3 ...,230,0.735271
1,Hombres,12-17 años,Bachillerato incompleto (1 a 2 años),670,2.141875
2,Hombres,12-17 años,Estudios Universitarios completos (4 a 5 años),1,0.003197
3,Hombres,12-17 años,Estudios Universitarios incompletos (1 a 3 años),14,0.044756
4,Hombres,12-17 años,Estudios de Posgrado (2 a 4 años),0,0.000000
...,...,...,...,...,...
75,Mujeres,60-75 años,No sabe/No contesta,17,0.054346
76,Mujeres,60-75 años,Primaria completa (6 años),899,2.873949
77,Mujeres,60-75 años,Primaria incompleta (1 a 5 años),715,2.285733
78,Mujeres,60-75 años,Secundaria completa o equivalente (3 años),636,2.033183


# Estatal

In [51]:
agrupado_por_entidad_sexo = b.groupby(['clave_entidad_dai', 'entidad', 'sexo', 'nivel_educativo']).size().reset_index(name='conteo')

agrupado_por_entidad_sexo['porcentaje'] = agrupado_por_entidad_sexo.groupby(['entidad', 'sexo'])['conteo'].transform(lambda x: (x / x.sum()) * 100)
agrupado_por_entidad_sexo

,clave_entidad_dai,entidad,sexo,nivel_educativo,conteo,porcentaje
0,01,Aguascalientes,Hombres,Bachillerato completo o equivalente (aprox. 3 ...,78,17.333333
1,01,Aguascalientes,Hombres,Bachillerato incompleto (1 a 2 años),44,9.777778
2,01,Aguascalientes,Hombres,Estudios Universitarios completos (4 a 5 años),74,16.444444
3,01,Aguascalientes,Hombres,Estudios Universitarios incompletos (1 a 3 años),18,4.000000
4,01,Aguascalientes,Hombres,Estudios de Posgrado (2 a 4 años),15,3.333333
...,...,...,...,...,...,...
602,32,Zacatecas,Mujeres,No sabe/No contesta,2,0.253485
603,32,Zacatecas,Mujeres,Primaria completa (6 años),95,12.040558
604,32,Zacatecas,Mujeres,Primaria incompleta (1 a 5 años),44,5.576679
605,32,Zacatecas,Mujeres,Secundaria completa o equivalente (3 años),264,33.460076


# Regional

In [52]:
agrupado_por_region_sexo = b.groupby(['region', 'sexo', 'nivel_educativo']).size().reset_index(name='conteo')
agrupado_por_region_sexo['porcentaje'] = agrupado_por_region_sexo.groupby(['region', 'sexo'])['conteo'].transform(lambda x: (x / x.sum()) * 100)
agrupado_por_region_sexo

,region,sexo,nivel_educativo,conteo,porcentaje
0,CDMX,Hombres,Bachillerato completo o equivalente (aprox. 3 ...,103,19.180633
1,CDMX,Hombres,Bachillerato incompleto (1 a 2 años),59,10.986965
2,CDMX,Hombres,Estudios Universitarios completos (4 a 5 años),99,18.435754
3,CDMX,Hombres,Estudios Universitarios incompletos (1 a 3 años),45,8.379888
4,CDMX,Hombres,Estudios de Posgrado (2 a 4 años),21,3.910615
...,...,...,...,...,...
154,Sur,Mujeres,No sabe/No contesta,3,0.133511
155,Sur,Mujeres,Primaria completa (6 años),350,15.576324
156,Sur,Mujeres,Primaria incompleta (1 a 5 años),170,7.565643
157,Sur,Mujeres,Secundaria completa o equivalente (3 años),626,27.859368
